# Decision Tree Implementation from Scratch

### Import

In [7]:
import numpy as np
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import argparse
from sklearn.datasets import load_breast_cancer, load_iris

### Node

In [8]:
class Node:
    
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

### Gini Impurity Coefficient
Compute Gini impurity of a non-empty node.Gini impurity is defined as Σ p(1-p) over all classes, with p the frequency of a class within the node. Since Σ p = 1, this is equivalent to 1 - Σ p^2.

In [9]:
def gini_coefficient(y):
    n_classes = len(set(y))
    m = y.size
    return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in range(n_classes))

### Fit
This function starts the recursive algorithm that will grow our tree, where each node is a a point where the tree split, based on a threshold.

In [10]:
def fit(X,y):
    n_classes = len(set(y))
    n_features = X.shape[1]
    tree = grow_tree(X, y)
    return tree

### Predict
Prediction is done my just applying the threshold at each node and deciding which child node (left or right) should be considered next

In [11]:
def predict(tree,x):
    
    feauture_path = []
    node = tree
    while node.left:
        feauture_path.append(node.feature_index)
        if x[node.feature_index] < node.threshold:
            node = node.left
        else:
            node = node.right
    return node.predicted_class,feauture_path

### Find Best Split
Find the best split for a node. "Best" means that the average impurity of the two children, weighted by their population, is the smallest possible. Additionally it must be less than the impurity of the current node. To find the best split, we loop through all the features, and consider all the midpoints between adjacent training samples as possible thresholds. We compute the Gini impurity of the split generated by that particular feature/threshold pair, and return the pair with smallest impurity.

In [1]:
def find_best_split(X, y):
    
    n_classes = len(set(y))
    n_features = X.shape[1]
    m = y.size
    if m <= 1 or n_classes <= 1:
        return None, None

    #Count of each class in the current node.
    num_parent = [np.sum(y == c) for c in range(n_classes)]

    #Gini of current node.
    best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
    best_idx, best_thr = None, None

    #Loop through all features.
    for idx in range(n_features):

        # Sort data along selected feature.
        thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
    
        # We could actually split the node according to each feature/threshold pair
        # and count the resulting population for each class in the children, but
        # instead we compute them in an iterative fashion, making this for loop
        # linear rather than quadratic.
        num_left = [0] * n_classes
        num_right = num_parent.copy()
        for i in range(1, m):  # possible split positions
            c = classes[i - 1]
            num_left[c] += 1
            num_right[c] -= 1
            gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in range(n_classes))
            gini_right = 1.0 - sum((num_right[x] / (m - i)) ** 2 for x in range(n_classes))

            #The Gini impurity of a split is the weighted average of the Gini impurity of the children.
            gini = (i * gini_left + (m - i) * gini_right) / m

            #The following condition is to make sure we don't try to split two
            #points with identical values for that feature, as it is impossible
            #(both have to end up on the same side of a split).
            if thresholds[i] == thresholds[i - 1]:
                continue

            if gini < best_gini:
                best_gini = gini
                best_idx = idx
                best_thr = (thresholds[i] + thresholds[i - 1]) / 2

    return best_idx, best_thr

### Grow Tree Recursively
Build a decision tree by recursively finding the best split.

In [13]:
def grow_tree(X, y, depth=0):
    
    #Create a Node
    n_classes = len(set(y))
    n_features = X.shape[1]
    num_samples_per_class = [np.sum(y == i) for i in range(n_classes)]
    predicted_class = np.argmax(num_samples_per_class)
    node = Node(
        gini=gini_coefficient(y),
        num_samples=y.size,
        num_samples_per_class=num_samples_per_class,
        predicted_class=predicted_class)

    #Split recursively until maximum depth is reached.
    if depth < max_depth:
        idx, thr = find_best_split(X, y)
        if idx is not None:
            indices_left = X[:, idx] < thr
            X_left, y_left = X[indices_left], y[indices_left]
            X_right, y_right = X[~indices_left], y[~indices_left]
            node.feature_index = idx
            node.threshold = thr
            node.left = grow_tree(X_left, y_left, depth + 1)
            node.right = grow_tree(X_right, y_right, depth + 1)
    return node

### Load Data and Init

In [14]:
dataset = load_breast_cancer()
X, y = dataset.data, dataset.target
max_depth = 5

### Train

In [15]:
tree = fit(X, y)

### Predict

In [16]:
y_pred,feauture_path = predict(tree,X[10,:])
print(feauture_path)
print(y_pred)

[20, 1, 24, 26, 1]
0
